In [1]:
import pandas as pd, numpy as np 
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [56]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)
valid = valid.reset_index()
labels = train.columns[2:]
ys_valid = valid[labels]

## COMBINE TOXIC CATEGORIES
y_valid = ys_valid.sum(axis=1)
y_valid.loc[y_valid>1] = 1

In [57]:
## Load SVM, XGBoost, and GRU results
NBSVM = pd.read_csv('../artifacts/simple/preds/tfidf_nbsvm.csv')
LSTM = pd.read_csv('../artifacts/simple/preds/glove_lstm.csv')

In [58]:
preds = (LSTM['glove_gru'] + NBSVM['tfidf_nbsvm'])/2
preds[preds<=0.5] = 0
preds[preds>0.5] = 1

In [59]:
# Evaluate predictions
results = pd.DataFrame(columns=['Label','Accuracy', 'Recall', 'Precision', 'F1', 'Vectorizer', 'model'])
acc, prec, recall, f1 = (accuracy_score(y_valid, preds), 
                            precision_score(y_valid, preds), 
                            recall_score(y_valid, preds), 
                            f1_score(y_valid, preds))

results = results.append({'Label': 'Toxic_Combined',
                        'Accuracy':acc,
                        'Recall':recall,
                        'Precision':prec,
                        'F1':f1,
                        'Vectorizer':'tfidf',
                        'model': 'NB_SVM'}, 
                        ignore_index = True)

print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                'Toxic_combined', 
                                acc, 
                                prec, 
                                recall,
                                f1))

Results for Toxic_combined comments: Accuracy - 0.97; Precision - 0.88; Recall - 0.78; F1 - 0.83


### Take a look at top 5 predictions for toxic comments, and top 5 predictions for non-toxic comments

In [60]:
preds = (LSTM['glove_gru'] + NBSVM['tfidf_nbsvm'])/2

In [94]:
top_toxic_idx = preds.sort_values(ascending=False)[:5].index.values
top_regular_idx = preds.sort_values()[:5].index.values

In [95]:
valid.loc[top_toxic_idx]['comment_text']

26829    yo fuck you bitch fuck you bitch fuck you bitc...
30281    eat shit and die bitch ass nigger eat shit and...
40695    fuck you ass hole go fuck yourself dirty fuck ...
31212    you a cunt your a fuck ass hole bitch il fuck ...
50087        burtalbert you fuck suck dick u king ass hole
Name: comment_text, dtype: object

In [96]:
for i in valid.loc[top_regular_idx]['comment_text']:
    print(i)

redirect talk ghar sansar film
redirect talk unterseeboot
redirect talk moulay idriss zerhoun
may thanks
redirect talk gj c


### Take a look at the comments with near uncertain probabilities

In [91]:
uncertain_toxic_idx = preds[(preds>0.45) & (preds<0.55)].sort_values(ascending=False)[:5].index.values
uncertain_regular_idx = preds[(preds>0.45) & (preds<0.55)].sort_values()[:5].index.values

In [92]:
for i in valid.loc[uncertain_toxic_idx]['comment_text']: print(i)

seriously though grow up flooding my user page with hundreds of question marks and saying byes certainly proves that you are an immature child ha ha byes xd
icbittc tnx re edgar i gave myself one last random page before retiring and it's clear that if i don't prioritize better i'm going to have to promote icbittc as an xtlc for i can't believe i typed that crap grin blush
thanks for calling me a zombie mind if i eat your brain just because i don't edit much doesn't make me a zombie account talk contrib
blood will be on your hands too jimbo blood will be on your hands too jimbo
nba finals and adam morrison you are annoying and ruining other people's fun please stop deleting all the edits on the above pages you are nothing but a buzzkill who is pissing other people off


In [93]:
for i in valid.loc[uncertain_regular_idx]['comment_text']: print(i)

don't hide the incriminating photographs you nazi americans if the photos are graphic its because the horrible deeds committed by the deranges sadistic perverted soldiers are also equally disturing i think the yanks should stop trying to cover up these issues
joppa and regarding your final warnings you seem to think you've got more power than you actually have this is wikipedia sweetheart not the united states you can ban me from this site all you please i was only here to defend i told you during our debate i couldn't give a shit whether or not my site was on here it was the people who created the article i was worried about you've already destroyed that
dear gracious and benevolent companion from looking back at my little smilely face i can understand why you though it looked like a penis obviously we have different views and i am quite sorry for not keeping a neutral view and telling you that the smiely face is actually a smiely face i hope you understand
ok i concede that relatives